## 搭建 ResNet34 网络

In [1]:
from torch import nn
import torch as t
from torch.nn import functional as F
from torch.autograd import Variable as V

In [2]:
class ResidualBlock(nn.Module):
    """
    实现子module: Residual Block
    """
    def __init__(self, inchannel, outchannel, stride=1, shortcut=None):
        super(ResidualBlock, self).__init__()
        self.left = nn.Sequential(
            nn.Conv2d(in_channels=inchannel, out_channels=outchannel, kernel_size=3, stride=stride, padding=1, bias=False),
            nn.BatchNorm2d(outchannel),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=outchannel, out_channels=outchannel, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(outchannel)
        )
        self.right = shortcut
    
    def forward(self, x):
        out = self.left(x)
        residual = x if self.right is None else self.right(x)
        out += residual
        return F.relu(out)

In [3]:
class ResNet(nn.Module):
    """
    实现主module: ResNet34
    ResNet34 包含多个layer, 每个layer又包含多个residual block
    用子module来实现residual block, 用 _make_layer 函数来实现layer
    """
    def __init__(self, num_classes=1000):
        super(ResNet, self).__init__()
        # 前几层图像转换
        self.pre = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=64, kernel_size=7, stride=2, padding=3, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        )
        
        # 重复的layer, 分别有3, 4, 6, 3个residual block
        self.layer1 = self._make_layer(64, 64, 3)
        self.layer2 = self._make_layer(64, 128, 4, stride=2)
        self.layer3 = self._make_layer(128, 256, 6, stride=2)
        self.layer4 = self._make_layer(256, 512, 3, stride=2)
        
        # 分类用的全连接
        self.fc = nn.Linear(512, num_classes)
    
    def _make_layer(self, inchannel, outchannel, block_num, stride=1):
        """
        构建layer, 包含多个residual block
        """
        shortcut = nn.Sequential(
            nn.Conv2d(in_channels=inchannel, out_channels=outchannel, kernel_size=1, stride=stride, bias=False),
            nn.BatchNorm2d(outchannel)
        )
        layers = []
        layers.append(ResidualBlock(inchannel, outchannel, stride, shortcut))
        
        for i in range(1, block_num):
            layers.append(ResidualBlock(outchannel, outchannel))
        return nn.Sequential(*layers)
    
    def forward(self, x):
        x = self.pre(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = F.avg_pool2d(x, 7)
        x = x.view(x.size(0), -1)
        return self.fc(x)

In [4]:
model = ResNet()
input = V(t.randn(1, 3, 224, 224))
out = model(input)
out

tensor([[-0.5078, -0.0444, -0.1475, -0.6192,  0.6766, -0.3655,  0.4035,  0.2271,
         -0.3594,  0.1724,  0.0050, -0.4060,  0.4394,  0.2365, -0.1953, -0.4293,
         -0.3881, -0.4816, -0.1599, -0.1557,  0.2584, -0.4328,  0.1026, -0.0159,
          0.0008, -0.1843,  0.2043, -0.1995, -0.1417, -0.5246,  0.1327, -0.0158,
         -0.0346,  0.1849, -0.1738, -0.4253, -0.0453, -0.1171,  0.1861, -0.2695,
         -0.5154, -0.4277,  0.4693,  0.2242, -0.0932,  0.0968, -0.1271, -0.3864,
          0.1029, -0.5248, -0.0107,  0.2686,  0.7145, -0.1095,  0.2055,  0.0953,
         -0.7000, -0.1644, -0.0966, -0.0216,  0.3671,  0.2210,  0.6721, -0.2888,
         -0.3524, -0.1813,  0.3751, -0.2300, -0.0895, -0.5230,  0.0012,  0.0267,
         -0.3087, -0.3349, -0.3700,  0.1403, -0.2747,  0.0942,  0.4063,  0.0157,
          0.3292, -0.2051,  0.0263, -0.3033,  0.5155, -0.2181,  0.5700,  0.1077,
          0.1490, -0.2801,  0.2373, -0.0859,  0.1237, -0.2202, -0.4983,  0.3384,
          0.0907, -0.2956, -